In [2]:
import os

current_dir = r"C:\Users\acer\Desktop\TMJOA\Preprocessing\temp0"

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

print(f"Parent directory: {parent_dir}")

Parent directory: C:\Users\acer\Desktop\TMJOA\Preprocessing


In [1]:
import os

# Replace this with the path to your main folder
root_folder = "D:\Drive D\models"

# Get all files ending with "lr0001.h5" in all subfolders
model_paths = [
    os.path.join(dirpath, filename)
    for dirpath, dirnames, filenames in os.walk(root_folder)
    for filename in filenames
    if filename.endswith("lr0001.h5")
]

# Now you can iterate through these files
print(f"There {len(model_paths)} models in {root_folder}")
for file_path in model_paths:
    print(file_path)
    # Add your processing code here

There 12 models in D:\Drive D\models
D:\Drive D\models\EfficientNetB5\EfficientNetB5_bo20_lr0001.h5
D:\Drive D\models\EfficientNetB7\EfficientNetB7_bo20_lr0001.h5
D:\Drive D\models\EfficientNetV2L\EfficientNetV2L_bo20_lr0001.h5
D:\Drive D\models\InceptionResNetV2\InceptionResNetV2_bo20_lr0001.h5
D:\Drive D\models\InceptionV3\InceptionV3_bo20_lr0001.h5
D:\Drive D\models\MobileNetV3Large\MobileNetV3Large_bo20_lr0001.h5
D:\Drive D\models\RegNetX320\RegNetX320_bo20_lr0001.h5
D:\Drive D\models\RegNetY320\RegNetY320_bo20_lr0001.h5
D:\Drive D\models\ResNet152\ResNet152-Bestof20_lr0001.h5
D:\Drive D\models\ResNet152V2\ResNet152V2_bo20_lr0001.h5
D:\Drive D\models\VGG19\VGG19_bo20_lr0001.h5
D:\Drive D\models\Xception\Xception_bo20_lr0001.h5


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
import pandas as pd
import time
import numpy as np

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

data_dir = r"C:\Users\kanan\Desktop\data_x\data_x\test"
test = tf.keras.utils.image_dataset_from_directory(data_dir, image_size=(224, 224))

Found 7360 files belonging to 2 classes.


In [3]:
results = []

for model_path in model_paths:

    test_model = load_model(model_path)
    test_iterator = test.as_numpy_iterator()
    batch = test_iterator.next()

    inference_times = []
    for batch in test_iterator: 
        X, y = batch
        start_time = time.time()
        yhat = test_model.predict(X)
        end_time = time.time()
        inference_times.append(end_time - start_time)
        pre.update_state(y, yhat)
        re.update_state(y, yhat)
        acc.update_state(y, yhat)
    
    avg_inference_time = np.mean(inference_times)
    f1_score = 2 * (pre.result() * re.result()) / (pre.result() + re.result())

    model_name = os.path.basename(model_path)

    results.append({
        'model_name': model_name,
        'accuracy': acc.result().numpy(),
        'precision': pre.result().numpy(),
        'recall': re.result().numpy(),
        'f1_score': f1_score.numpy(),
        'avg_inference_time': avg_inference_time
    })

df_results = pd.DataFrame(results)

1/1 [==============================] - 0s 25ms/step


In [4]:
print(df_results)

                          model_name  accuracy  precision    recall  f1_score  \
0      EfficientNetB5_bo20_lr0001.h5  0.779885   0.860413  0.766741  0.810880   
1      EfficientNetB7_bo20_lr0001.h5  0.780568   0.854197  0.775992  0.813219   
2     EfficientNetV2L_bo20_lr0001.h5  0.775701   0.853562  0.767361  0.808170   
3   InceptionResNetV2_bo20_lr0001.h5  0.735501   0.763861  0.825463  0.793468   
4         InceptionV3_bo20_lr0001.h5  0.711517   0.723355  0.860373  0.785937   
5    MobileNetV3Large_bo20_lr0001.h5  0.720501   0.740133  0.841248  0.787457   
6          RegNetX320_bo20_lr0001.h5  0.730622   0.755726  0.830889  0.791527   
7          RegNetY320_bo20_lr0001.h5  0.736797   0.765662  0.824783  0.794124   
8       ResNet152-Bestof20_lr0001.h5  0.741524   0.774040  0.819176  0.795969   
9         ResNet152V2_bo20_lr0001.h5  0.728889   0.750890  0.837251  0.791722   
10              VGG19_bo20_lr0001.h5  0.734803   0.760665  0.830465  0.794034   
11           Xception_bo20_l

In [5]:
save_folder = r"C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Training logs"
excel_filename = r"model_evaluation_results.xlsx"
save_path = os.path.join(save_folder,excel_filename)

counter = 1
while os.path.exists(save_path):
    excel_filename = f'model_evaluation_results_{counter}.xlsx'
    save_path = os.path.join(save_folder,excel_filename)
    counter += 1

df_results.to_excel(save_path, index=False)
print(f"Results saved to {save_path}")

Results saved to C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Training logs\model_evaluation_results.xlsx
